In [2]:
import os

# import
import pandas as pd

In [ ]:
# import files 
main_df = pd.read_csv('../PM2.5_Vehicles_Population.csv')
df_pop = pd.read_csv('../California_Population.csv') 

In [ ]:
main_df
# main_df['Population Z Score'] = main_df['Population Z Score'].round(2)

In [ ]:
# total regestered car users/population 
def create_comparison(county):
    df = main_df[main_df['County'] == county]
    df.loc[:,'Total Cars'] = df['Number of Gas Cars'] + df['Number of EV']
    df.loc[:,'Total Cars/Population'] = (df['Total Cars'] / df['Total Population']).round(2)
    df.loc[:,'EV/Total Cars'] = (df['Number of EV'] / df['Total Cars']).round(2)
    df.loc[:,'Gas/Total Cars'] = (df['Number of Gas Cars'] / df['Total Cars']).round(2)
    return df[['Total Population Z Score','Total Cars/Population', 'EV/Total Cars', 'Gas/Total Cars']] 


In [ ]:
# so as more people got cars, more ev were regestered. 
# what is the relationship between population growth and ev adaptation?
## it seems to be positive, direct relationship 
## so as more population increases, ev car adaption also increases. i wonder if this is the same across counties? 

In [1]:
san_diego = create_comparison('San Diego')
san_diego

NameError: name 'create_comparison' is not defined

In [ ]:
 # los_angeles = create_comparison('Los Angeles')
# los_angeles

In [ ]:
# orange = create_comparison('Orange')
# orange

In [ ]:
# san_fran =  create_comparison('San Francisco')
# san_fran

In [ ]:
# mono = create_comparison('Mono')
# mono 

In [ ]:
# is this crazy?
# function takes dataset from online and turns it into a merge-able df for main_df 

In [ ]:
def aged_population(dataset, datayear):
    # load files 
    df = pd.read_csv(dataset)
    df = df[df['Label (Grouping)'].str.strip() == '18 years and over']
    df = df[df['Label (Grouping)'].str.contains('18 years and over', case=False, na=False)]
    # extract unique county names
    county_names = [col.split("!!")[0] for col in df.columns if "California" in col]
    county_names = list(set(county_names))
    # extract county total and margin of error from main dataset
    county_total = [f"{county}!!Total!!Estimate" for county in county_names]
    county_margin_of_error = [f"{county}!!Total!!Margin of Error" for county in county_names]
    # making df of those
    county_total = df.loc[:, county_total]
    county_margin_of_error = df.loc[:, county_margin_of_error]
    # creating county total population estimate df 
    ct_df =county_total.transpose()
    ct_df = ct_df.reset_index() 
    # some years record >21, others >18. 
    ct_df.columns = ['County' , 'Population Estimate, >18']
    ct_df['County'] = ct_df['County'].str.split(' County').str[0]
    ct_df = ct_df.sort_values(by='County', ascending=True).reset_index(drop=True)
    
    # creating county margin of error 
    me_df = county_margin_of_error.transpose()
    me_df = me_df.reset_index() 
    me_df.columns = ['County' , 'Population Margin of Error, >18']
    me_df['County'] = me_df['County'].str.split(' County').str[0]
    me_df = me_df.sort_values(by='County', ascending=True).reset_index(drop=True) 
    
    # merging on county 
    ct_me_df = pd.merge(ct_df, me_df, how='inner', on = ['County'])
    ct_me_df['Data Year'] = datayear
    return ct_me_df

In [ ]:
# run function to create mergable df
totpop2010 = aged_population('./Pop_Over_21_2010.S0101-2024-09-27T210322.csv', 2010)
totpop2011 = aged_population('./Pop_Over_21_2011.S0101-2024-09-27T210305.csv', 2011)
totpop2012 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2012)
totpop2013 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2013)
totpop2014 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2014)
totpop2015 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2015)
totpop2016 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2016)
totpop2017 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2017)
totpop2018 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2018)
totpop2019 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2019)
totpop2020 = aged_population('./pop_over_18_2020.csv', 2020)
totpop2021 = aged_population('./Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2021)
totpop2022 = aged_population('./Pop_Over_21_2022.csv', 2022)
totpop2023 = aged_population('./Pop_Over_21_2023.S0101-2024-09-27T205616.csv', 2023)
# stacking totpop
dfs = [totpop2010, totpop2011, totpop2012, totpop2013, totpop2014, totpop2015, totpop2016, totpop2017, totpop2018, totpop2019, totpop2021, totpop2022, totpop2023]
aged_pop_2010_2023 = pd.concat(dfs, ignore_index= True)

In [ ]:
aged_pop_2010_2023['County'] = aged_pop_2010_2023['County'].str.strip()
counties = pd.DataFrame({
    'County_main_df': main_df['County'],
    'County_aged_pop_2010_2023': aged_pop_2010_2023['County']
})
counties.head()
# Get unique counties from both DataFrames
main_df_co = set(main_df['County'])
aged_pop_2010_2023_co = set(aged_pop_2010_2023['County'])

# Find differences
only_in_main_df_co = main_df_co - aged_pop_2010_2023_co
only_in_aged_pop_2010_2023_co = aged_pop_2010_2023_co - main_df_co

print("Only in main_df:", only_in_main_df_co)
print("Only in aged_pop_2010_2023:", only_in_aged_pop_2010_2023_co)

In [ ]:
# merging aged_pop_2010_2023 to main_df 
aged_pop_df = pd.merge(main_df, aged_pop_2010_2023, how='inner', on = ['Data Year', 'County'])

In [ ]:
# there is some percentages and whole numbers, let's make them all whole numbers 
def convert_to_population_estimate(row):
    # converting percentage to decimal, if applicable 
    if isinstance(row['Population Estimate, >18'], str) and '%' in row['Population Estimate, >18']:
        percent_value = float(row['Population Estimate, >18'].strip('%')) / 100
        return percent_value * row['Total Population']
    else:
        return row['Population Estimate, >18']

# apply to update 'Total Population Estimate, >21'
aged_pop_df['Population Estimate, >18'] = aged_pop_df.apply(convert_to_population_estimate, axis=1)



aged_pop_df = aged_pop_df[['County', 'Data Year', 'Total Population', 'Total Population Z Score', 
                           'Population Estimate, >18', 'Population Margin of Error, >18', 
                           'Number of Gas Cars', 'Number of EV'
                           ]]

aged_pop_df

In [ ]:
# to csv
aged_pop_df.to_csv('Main_DF_PM2.5_Vehicle_Pop.csv', index=False)


In [1]:
import pandas as pd 

In [2]:
csv_2010 = '../pm_data/ad_viz_plotval_data_2010.csv'
csv_2011 = 'pm_data/ad_viz_plotval_data_2011.csv'
csv_2012 = 'pm_data/ad_viz_plotval_data_2012.csv'
csv_2013 = 'pm_data/ad_viz_plotval_data_2013.csv'
csv_2014 = 'pm_data/ad_viz_plotval_data_2014.csv'
csv_2015 = 'pm_data/ad_viz_plotval_data_2015.csv'
csv_2016 = 'pm_data/ad_vis_plotval_data_2016.csv'
csv_2017 = 'pm_data/ad_viz_plotval_data_2017.csv'
csv_2018 = 'pm_data/ad_viz_plotval_data2018.csv'
csv_2019 = 'pm_data/ad_viz_plotval_data2019.csv'
csv_2020 = 'pm_data/ad_viz_plotval_data2020.csv'
csv_2021 = 'pm_data/ad_viz_plotval_data2020.csv'
csv_2022 = 'pm_data/ad_viz_plotval_data2022.csv'
csv_2023 = 'pm_data/ad_viz_plotval_data2023.csv'
csv_2024 = 'pm_data/ad_viz_plotval_data2024.csv'

In [9]:
# This function takes a file name, extracts applicable columns, 
# manipulates them into the proper format, and returns a mergeable dataframe. 
def avg_pm_value(filename): 
    # loading data
    df = pd.read_csv(filename)
    # We are only concerned with the following columns: 
    df = df[['Date', 'Daily Mean PM2.5 Concentration','Units', 'Site ID', 'County', 'Site Latitude', 'Site Longitude']]
    # changing date column to datetime 
    df['Date'] = pd.to_datetime(df['Date'])
    # adding year column 
    df['Data Year'] = df['Date'].dt.year
    # determining unique sites per county 
    unique_sites_per_county = df.groupby('County')['Site ID'].nunique().reset_index()
    unique_sites_per_county.rename(columns={'Site ID': 'Number of Site IDs/County'}, inplace=True) 
    # location average 
    average_latitude_county = df.groupby('County')['Site Latitude'].mean().reset_index()
    average_longitude_county =df.groupby('County')['Site Longitude'].mean().reset_index()
    county_longitude_latitude = average_latitude_county.merge(average_longitude_county, on='County')
    county_longitude_latitude.rename(columns={'Site Latitude': 'County Latitude', 'Site Longitude': 'County Longitude'}, inplace=True)
    # average pm2.5 values per county 
    avg_pm_value =df.groupby(['County', 'Data Year'])['Daily Mean PM2.5 Concentration'].mean().reset_index()
    avg_pm_value.rename(columns={'Daily Mean PM2.5 Concentration': 'Average PM 2.5/County'}, inplace=True)
    # merging tables 
    a_df = unique_sites_per_county.merge(avg_pm_value, on='County')
    avg_df = a_df.merge(county_longitude_latitude, on='County')
    avg_df = avg_df.pivot_table(index ='County', values = ['Data Year','Average PM 2.5/County', 'Number of Site IDs/County', 'County Latitude', 'County Longitude'])
    avg_df.reset_index(inplace=True)
    return avg_df

In [10]:
df23 = avg_pm_value(csv_2010)
df23

,County,Average PM 2.5/County,County Latitude,County Longitude,Data Year,Number of Site IDs/County
0,Alameda,8.165623,37.740259,-122.101771,2010,5
1,Butte,8.776451,39.501933,-121.689305,2010,3
2,Calaveras,7.380349,38.201850,-120.680277,2010,1
3,Colusa,6.054913,39.189190,-121.998870,2010,1
4,Contra Costa,7.606920,37.936013,-122.026154,2010,1
5,Del Norte,2.786869,41.560952,-124.083964,2010,1
6,El Dorado,2.195902,38.976000,-120.103506,2010,1
7,Fresno,11.729538,36.670999,-119.895476,2010,6
8,Glenn,8.386236,39.533870,-122.190834,2010,1
9,Humboldt,5.903409,40.789422,-124.170696,2010,2
